In [29]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    ElementNotInteractableException
)
import time

# Path to your CSV file
input_file_path = "input_file_path"
output_file_path = "output_file_path"

# Read the CSV file
df = pd.read_csv(input_file_path)

# Initialize Selenium WebDriver
chrome_options = Options()
# Uncomment the headless argument if you want to see the browser window
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--incognito")  # Incognito mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"
)

# Specify the path to your chromedriver
chromedriver_path = '/Users/victoralembik/Desktop/Downloads/chromedriver-mac-arm64/chromedriver'

# Use the specified path for ChromeDriver
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

def accept_cookies():
    """
    Accept cookies on the Facebook homepage if the prompt is shown.
    """
    driver.get("https://www.facebook.com")
    time.sleep(5)  # Allow time for the cookies prompt to appear
    try:
        # Find and click the "Allow essential and optional cookies" button
        cookies_buttons = driver.find_elements(By.XPATH, "//button[contains(string(), 'Allow essential and optional cookies')]")
        if cookies_buttons:
            cookies_buttons[0].click()
            print("Cookies accepted.")
        time.sleep(2)  # Wait for the page to adjust after accepting cookies
    except (NoSuchElementException, TimeoutException, ElementNotInteractableException) as e:
        print(f"Cookies prompt not found or not interactable: {e}")

def get_redirected_url(profile_link):
    """
    Visit the Facebook profile link and get the new URL after any potential redirects.
    """
    try:
        driver.get(profile_link)
        time.sleep(5)  # Allow time for the page to load and redirects to occur

        # Get the new URL from the address bar
        new_url = driver.current_url
        print(f"Original URL: {profile_link}")
        print(f"New URL: {new_url}")

        return new_url

    except TimeoutException:
        print(f"Timeout when processing {profile_link}")
        return profile_link
    except Exception as e:
        print(f"Error processing {profile_link}: {e}")
        return profile_link

# Step 1: Accept cookies on the Facebook homepage
accept_cookies()

# Step 2: Visit each facebook_profile_link and capture the redirected URL
df['facebook_profile_link'] = df['facebook_profile_link'].apply(get_redirected_url)

# Save the updated links to a new CSV file
df.to_csv(output_file_path, index=False)

# Clean up
driver.quit()

print(f"Corrected links saved to {output_file_path}")
